In [1]:
import os
import cv2
import numpy as np
import math
import natsort 
import fnmatch
from PIL import Image
import subprocess

In [2]:
path = "/home/m/Documents/cartoon/frames"
files = fnmatch.filter(os.listdir(path), '*.jpg') # get all the .jpg files

In [3]:
image_files = natsort.natsorted(files,reverse=False) # to sort out all the files in acending order

In [4]:
down_sample = up_sample = 2       # downsampling/upsampling applies twice

In [5]:
for file in image_files:
    img = cv2.imread(f"/home/m/Documents/cartoon/frames/{file}")
    
    im_color = img
    
    for _ in range(down_sample):
        im_color = cv2.pyrDown(im_color)
        
    """Apply average filtering"""
    kernel = np.ones((5,5),np.float32)/math.pi #define kernal size
    im_color = cv2.filter2D(im_color,-1,kernel)
    
    """upsampling the image to the original size"""
    for _ in range(up_sample):
        im_color = cv2.pyrUp(im_color)
    
    """Convert RGB to GARY scale"""
    im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    """Apply gaussian blur"""
    im_blur = cv2.GaussianBlur(im_gray, (5,5), cv2.BORDER_DEFAULT) 
    
    #  Apply adaptive thresholding for the image
    im_edge = cv2.adaptiveThreshold(im_blur, 255,
                                 cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY,
                                 blockSize=9,
                                 C=2)
    im_edge = cv2.cvtColor(im_edge, cv2.COLOR_GRAY2RGB) #Convert back to RGB
    cartoon = cv2.addWeighted(im_color,0.6,im_edge,0.4,0) # take two images & blend them together.
    
    save_to = '/home/m/Documents/cartoon/frames_cartoon'
    cv2.imwrite(os.path.join(save_to , f'{file}'), cartoon)
    cv2.waitKey(0)

In [6]:
save_to = "/home/m/Documents/cartoon/frames_cartoon" # path to save the resized images
irobot = 'iRobot_.mp4'

images = [im for im in os.listdir(save_to) if im.endswith(".jpg")]
images = natsort.natsorted(images,reverse=False) # to sort out all the files in acending order

In [7]:
frame = cv2.imread(os.path.join(save_to, images[0]))
height, width, channel = frame.shape

video = cv2.VideoWriter(irobot, cv2.VideoWriter_fourcc(*'XVID'), 30, (width,height))
for image in images:
    video.write(cv2.imread(os.path.join(save_to, image)))

cv2.destroyAllWindows()
video.release()